# **Assignment \#1**: Machine Learning MC886
University of Campinas (UNICAMP), Institute of Computing (IC)

Prof. Sandra Avila, 2020s2



## Objective 

Explore **linear regression** alternatives and come up with the best possible model to the problems, avoiding overfitting. In particular, predict the **price of diamonds** from their attributes (e.g., depth, clarity, color) using the Diamonds dataset (https://www.kaggle.com/shivam2503/diamonds)

## Dataset

The Diamonds dataset contains the prices and attributes of almost 50,000 diamonds.

Dataset Information: You should respect the following traininig/test split: 45,000 training examples, and 5,000 test examples.

There are 9 attributes as follows: 

- 1: **carat**: weight of the diamond (0.2-5.01)
- 2: **cut**: quality of the cut (Fair, Good, Very Good, Premium, Ideal)
- 3: **color**: diamond color, from J (worst) to D (best)
- 4: **clarity**: a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
- 5: **x**: length in mm (0-10.74)
- 6: **y**: width in mm (0-58.9)
- 7: **z**: depth in mm (0-31.8)
- 8: **depth**: total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43-79)
- 9: **table**: width of top of diamond relative to widest point (43-95)


target **price**: price in US dollars

The data is available at
https://www.dropbox.com/s/tmz8bkocrpfmfb9/diamonds-dataset.zip


## Deadline

Monday, October 12th 7 pm. 

Penalty policy for late submission: You are not encouraged to submit your assignment after due date. However, in case you did, your grade will be penalized as follows:
- October 13th 7 pm : grade * 0.75
- October 14th 7 pm : grade * 0.5
- October 15th 7 pm : grade * 0.25


## Submission

On Google Classroom, submit your Jupyter Notebook (in Portuguese or English).

**This activity is NOT individual, it must be done in pairs (two-person group).**

## Activities

1. (4 points) Perform Linear Regression. You should implement your solution and compare it with ```sklearn.linear_model.SGDRegressor``` (linear model fitted by minimizing a regularized empirical loss with SGD, http://scikit-learn.org). Keep in mind that friends don't let friends use testing data for training :-)



In [ ]:
# TODO: Load and preprocess your dataset.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

data = pd.read_csv("diamonds-train.csv", na_values = ['no info', '.'])
data = data.replace(['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'], [1, 2, 3, 4, 5])
data = data.replace(['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF'], [1, 2, 3, 4, 5, 6, 7, 8])
data = data.replace(['J', 'I', 'H', 'G', 'F', 'E', 'D'], [1, 2, 3, 4, 5, 6, 7])
data_norm = data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))

data_test = pd.read_csv("diamonds-test.csv", na_values = ['no info', '.'])
data_test = data_test.replace(['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'], [1, 2, 3, 4, 5])
data_test = data_test.replace(['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF'], [1, 2, 3, 4, 5, 6, 7, 8])
data_test = data_test.replace(['J', 'I', 'H', 'G', 'F', 'E', 'D'], [1, 2, 3, 4, 5, 6, 7])


In [ ]:
# TODO: Linear Regression. Implement your solution. You cannot use scikit-learn libraries.
def predict(x,theta):
    x=np.c_[np.ones(len(x)),x]
    return np.dot(x, theta)

def gradient_b(x, y, theta): 
    h = predict(x, theta)
    gradient = [0]*len(theta)
    for i in range(len(y)):
        gradient[0] += h[i] - y[i]
    for k in range(len(theta)-1):
        for j in range(len(y)):
            gradient[k+1] += (h[j] - y[j])*x[j][k]
    return gradient

def cost(x, y, theta): 
    h = predict(x, theta)
    J = 0
    for i in range(len(y)):
        J += (h[i]-y[i])**2
    J /= 2*len(y)
    return J

def linear_eq(data,alpha,epochs):
    error = []
    x = data[['carat','cut','color','clarity','x','y','z','depth','table']].values
    y = data['price'].values
    t = np.zeros((data.shape[1],1))
    for e in range(epochs):
        gradient = gradient_b(x,y,t)
        t[0] = t[0] - (alpha*gradient[0])/len(y)
        for j in range(1,len(t)):
            t[j] = t[j] - (alpha*gradient[j])/len(y)
        error.append(cost(x,y,t))
    return t, error

t,e=linear_eq(data,1e-4,20)
y = data_test['price'].values
x = data_test[['carat','cut','color','clarity','x','y','z','depth','table']].values
y_pred = predict(x,t)
#print(e[1]," vs. ", e[-1])
plt.scatter(y,y_pred)

In [ ]:
# TODO: Linear Regression. Implement your solution with sklearn.linear_model.SGDRegressor.
def sklearn_SGD(data): 
  from sklearn.linear_model import SGDRegressor
  from sklearn.pipeline import make_pipeline
  from sklearn.preprocessing import StandardScaler
 
  x = data[['carat','cut','color','clarity','x','y','z','depth','table']]
  y = data['price']
  model = make_pipeline(StandardScaler(), SGDRegressor(alpha=0.01,max_iter=5000, tol=1e-4))
  model.fit(x,y)
  predict = model.predict(x)
  return predict

t = sklearn_SGD(data)
plt.scatter(y,t)


> What are the conclusions? (1-2 paragraphs)

 Em comparação com o a implementação do Scikit-learn, nosso codigo foi bem mais custoso, tanto em execução, quanto no quesito erro. Muito por uma possivel má otimização na hora dos calculos (utilizamos for para o calculo de diversos elementos ao invés de fazer produto linear) tanto quanto na forma do modelo escolhido, já que este modelo é um modelo básico montado em um polinomio de primeira ordem.
 Quanto ao erro, testamos até 100 epocas, por causa do tempo de execução (muito alto se comparado com o SGD do scikit), tendo uma ideia de que o erro estava sendo atenuado, mas não conseguimos ver o quão bem ele poderia desempenhar com maiores iterações.



2. (2 points) Sometimes, we need some more complex function to make good prediction. Devise and test more complex model. 


In [ ]:
# TODO: Complex model. Implement your solution. You cannot use scikit-learn libraries.

def predict(x,theta):
    x=np.c_[np.ones(len(x)),x]
    return np.dot(x, theta)

def gradient_b(x, y, theta): 
    h = predict(x, theta)
    gradient = [0]*len(theta)
    for i in range(len(y)):
        gradient[0] += h[i] - y[i]
    for k in range(len(theta)-1):
        for j in range(len(y)):
            gradient[k+1] += (h[j] - y[j])*x[j][k]
    return gradient

def cost(x, y, theta): 
    h = predict(x, theta)
    J = 0
    for i in range(len(y)):
        J += (h[i]-y[i])**2
    J /= 2*len(y)
    return J

def matrix_union(A, B):
    for a, b in zip(A, B):
        yield [*a, *b]

def linear_eq_grau2(data,alpha,epochs):
    error = []
    x = data[['carat','cut','color','clarity','x','y','z','depth','table']].values
    y = data['price'].values
    t = np.zeros((data.shape[1]*2-1,1))
    c = x*x
    x = list(matrix_union(x,c))
    for e in range(epochs):
        error.append(cost(x,y,t))
        gradient = gradient_b(x,y,t)
        t[0] = t[0] - (alpha*gradient[0])/len(y)
        for j in range(1,len(t)):
            t[j] = t[j] - (alpha*gradient[j])/len(y)
    return t, error, x

t,e,x=linear_eq_grau2(data,0.0001,10)
y = data['price'].values
y_pred = predict(x,t)
#print(e[1]," vs. ", e[-1])
plt.scatter(y,y_pred)


> What are the conclusions? (1-2 paragraphs)

Observamos que houve um contraste em relação ao comportamento do erro quanto a função implementada de grau 1, neste modelo, o erro variou em certas faixas para um valor maior e em outras para menos. Mudamos os valores das iterações entre o intervalo [1-20] e vimos que entre os valores 5-8 foram apresentadas as melhores taxas de erro, mas ainda sim, para esse conjunto de dados, a nossa implementação mais simples, com todas as features em grau 1, obtivemos um resultado melhor, mas longe do esperado. Esta má otimização pode ter ocorrido devido a nossa escolha de colocar todas as features em um grau maior, ao inves, de analisar primeiro os dados e ver qual modelo polinomial melhor se encaixava em cada uma.

3. (1 point) Plot the cost function vs. number of iterations in the training set and analyze the model complexity. 

In [ ]:
# TODO: Plot the cost function vs. number of iterations in the training set.
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

t,e,x1=linear_eq_grau2(data,0.0001,10)

y = e
x = np.zeros(len(y))
for i in range(len(y)):
  x[i] = i+1

plt.xlabel('iteration')
plt.ylabel('error')

plt.plot(x, y, 'o', color='blue');



 > What are the conclusions? What are the actions after such analyses? (1-2 paragraphs)

 Observamos que em media, entre as iterações a taxa de erro variava pouco em comparação com iterações maiores, apartir de iterações maiores (10 ou mais), o erro explodia para casas muito altas, ficando claro que há um erro no modelo. A melhor faixa de valores para o erro, foi entre as iterações 5 e 8, mas o modelo linear mais simples teve melhor aproveitamento.

 Tais falhas podem ter sido geradas no modo de implementação, ou mesmo no modelo proposto, em que todas as features assumiam valores polinomiais de grau 2 em vez de analisarmos cada feature e tentar prever um polinomio especifico para a mesma.


4. (3 points) Use different Gradient Descent (GD) learning rates when optimizing. Compare the GD-based solutions with Normal Equation. You should implement your solutions. What are the conclusions?


In [ ]:
# TODO: Gradient Descent (GD) with different learning rates. Implement your solution. You cannot use scikit-learn libraries.

#Mini Batch
def create_mini_batch(data,batch_size):
    m_b = []
    x = data[['carat','cut','color','clarity','x','y','z','depth','table']].values
    y = data['price'].values
    batch = np.c_[x,y]
    np.random.shuffle(batch)
    number_of_batches = batch.shape[0]//batch_size
    i=0
    for i in range(number_of_batches+1):
        mini_batch = batch[i * batch_size:(i + 1)*batch_size, :] 
        x_m = mini_batch[:, :-1]
        y_m = mini_batch[:, -1].reshape((-1, 1))
        m_b.append((x_m, y_m))
    if data.shape[0] % batch_size != 0: 
        mini_batch = batch[i * batch_size:data.shape[0]] 
        x_mini = mini_batch[:, :-1] 
        y_mini = mini_batch[:, -1].reshape((-1, 1)) 
        m_b.append((x_mini, y_mini)) 
    return m_b

def predict(x,theta):
    x=np.c_[np.ones(len(x)),x]
    return np.dot(x, theta)

def gradient_mb(x, y, theta): 
    h = predict(x, theta)
    gradient = [0]*len(theta)
    for i in range(len(y)):
        gradient[0] += h[i] - y[i]
    for k in range(len(theta)-1):
        for j in range(len(y)):
            gradient[k+1] += (h[j] - y[j])*x[j][k]
    return gradient

def cost(x, y, theta): 
    h = predict(x, theta)
    J = 0
    for i in range(len(y)):
        J += (h[i]-y[i])**2
    J /= 2*len(y)
    return J

def mini_batch_gd(data,alpha,epochs,batch_size):
    error = []
    t = np.zeros((data.shape[1],1))
    for i in range(epochs):
        mini_batches = create_mini_batch(data,batch_size)
        for batch in mini_batches:
            x,y = batch
            gradient = gradient_mb(x,y,t)
            if len(y) > 0:
              t[0] = t[0] - (alpha*gradient[0])/len(y)
              for j in range(1,len(t)):
                  t[j] = t[j] - (alpha*gradient[j])/len(y)
              error.append(cost(x,y,t))
    return t, error

#SGD

def predict_sgd(x,theta):
    x=np.r_[1,x]
    return np.dot(x,theta)

def gradient_sgd(x, y, theta): 
    h = predict_sgd(x, theta)
    gradient = [0]*len(theta)
    gradient[0] = (h - y)
    for k in range(len(theta)-1):
        gradient[k+1] += (h - y)*x[k]
    return gradient

def cost_sgd(x, y, theta): 
    h = predict_sgd(x, theta)
    J = (h-y)**2
    J /= 2
    return J

def stochastic_gd(data,alpha,epochs):
    data_shuff = data.values
    error = []
    t = np.zeros((data.shape[1],1))
    for i in range(epochs):
        np.random.shuffle(data_shuff)
        for example in data_shuff:
            x = example[:-1]
            y = example[-1]
            gradient = gradient_sgd(x,y,t)
            t[0] = t[0] - (alpha*gradient[0])
            for j in range(1,len(t)):
              t[j] = t[j] - (alpha*gradient[j])
            error.append(cost_sgd(x,y,t))
    return t, error

x = data_test[['carat','cut','color','clarity','x','y','z','depth','table']].values
y = data_test['price'].values
t, error = stochastic_gd(data,0.0001,20)
#t, error = mini_batch_gd(data,0.0001,20,500):
y_pred = predict(x,t)
plt.scatter(y,y_pred)

In [ ]:
# TODO: Compare the GD-based solutions (e.g., Batch, SGD, Mini-batch) with Normal Equation. Implement your solution. You cannot use scikit-learn libraries.

def normal_eq(data):
  #Normal equation implementation
  x = data[['carat','cut','color','clarity','x','y','z','depth','table']].values
  y = data['price'].values
  x=np.c_[np.ones(len(y)),x]
  #Calculo do (x_transposto*x)^(-1)
  x_trans = np.transpose(x)
  x_trans_esc = x_trans.dot(x)
  #Calculo do (x_transposto*x)^(-1) * (x_transposto*y)
  c1 = np.linalg.inv(x_trans_esc)
  c2 = x_trans.dot(y)
  result = c1.dot(c2)
  return result

#Teste
result = normal_eq(data)
y_pred = predict(x,result)
plt.scatter(y,y_pred)

# > What are the conclusions? (2-4 paragraphs)

  Observando o comportamento das funções com GD Mini batch e Estocastico, temos que as duas só convergiram com uma learning rate menor que 1e-3, como o estocastico atualiza seu theta por cada iteração, obtivemos um resultado melhor para o erro por epoca em relação ao mini batch, porém exigiu um maior tempo para executa-lo.

  Comparando a execução dos dois GD com o metodo da Equação normal, temos que a equação normal obteve um resultado muito melhor que os outros dois, inclusive, tendo desempenho parecido com a função SGD implementada do Scikit Learn. Foi muito mais rapido, além de não ter problemas de divergencia com learning rate e ter um custo de processamento bem menor para o conjunto de dados usado, pois não precisa iterar.

  Então, para um conjunto de dados relativamente pequeno, a equação normal tende a ser uma melhor alternativa, porém, temos que levar em conta que é um processo de computação matricial, então, se tivermos uma matriz de ordem relativamente grande, teremos um processo mais lento, além de algumas matrizes não serem invertiveis, impossibilitando assim, seu calculo.
